![image-2.png](attachment:image-2.png)
_Aprendizaje Automático_

_Máster Universitario en Inteligencia Artificial_

# Actividad grupal: Explorando patrones ocultos: Detección de Anomalías en datos
## Objetivos

Mediante este trabajo se pretende que ponga en práctica la aplicación de las técnicas de detección de anomalías. El objetivo es analizar si existen anomalías en el conjunto de datos aplicando técnicas univariantes y multivariantes. Debe detallar los pasos que hay que realizar para detección de valores inusuales.

- 	Entender los métodos media móvil, z-score, Isolation Forest y Local Outlier Factor
- 	Implementar el código para cada una de estas técnicas y analizar si existen o no anomalía
- 
	Investigar sobre una técnica de detección de anomalías con un caso de uso concreto.

Descripción de la actividad

Debes realizar la actividad en el Notebook adjunto. Debes ir completando el código solicitado, así como responder a las preguntas que se van realizando.

Tareas que
- alizar
	Detección de 
- malías.
	Debes completar el código pedido en este notebook y contestar las pregunt
- finales.
	Buscar dos artículos científicos (https://scholar.google.es/) con un caso de uso de detección de anomalías. Los artículos deben estar en revistas científicas, y deben ser posteriores a 2015. No debe utilizar técnicas de De
- Learning.
	Para cada artí
- O indicar:
o	Objetivo: cuál es el objetivo de la investigación, es decir a qué problema real está aplicando la detección
-  anomalías.
o	Técnicas de detección de anoma
- s empleadas.
o	Principales resultados de la aplicación y de la investigación.


#### Integrantes del grupo

In [2]:
# Coloca en este espacio el nombre de los integrantes del grupo

## Detección de anomalías

Los transformadores juegan un papel muy importante en el sistema eléctrico. Aunque son algunos de los componentes más confiables de la red eléctrica, también son propensos a fallar debido a muchos factores, tanto internos como externos. Podría haber muchos iniciadores que causen una falla en el transformador, pero los que potencialmente pueden conducir a una falla catastrófica son los siguientes:
Falla mecánica
Falla dieléctrica

Contenido
Estos datos se recopilaron a través de dispositivos IoT desde el 25 de junio de 2019 hasta el 14 de abril de 2020 y se actualizaron cada 15 minutos.
El dataset contiene 19352 registros.
Descripción de los parámetros:
Voltaje de corriente:
1. VL1- Fase Línea 1
2. VL2- Fase Línea 2
3. VL3- Fase Línea 3
4. IL1- Línea actual 1
5. IL2- Línea actual 2
6. IL3- Línea actual 3
7. VL12- Línea de tensión 1 2
8. VL23- Línea de tensión 2 3
9. VL31- Línea de tensión 3 1
10. INUT-Corriente neutra

In [26]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# se carga el dataset, con separados ;, se transforman las dos columnas Date y Time a una única columna con tipo datetime
# en el fichero CSV existen NaN identificados por ?
df = pd.read_csv('CurrentVoltage.csv', sep=',') 

#### ¿Cuántas instancias tiene el dataset?

In [51]:
# Pon aquí tu código

#### ¿Cuál es el tipo de datos predominante en las variables?

In [52]:
#Inserta acá el código

## Variables nulas

#### ¿Tiene este dataset datos faltantes?

¿Tiene este dataset datos faltantes?

#### ¿Es necesario hacer algún tratamiendo adicional sobre los datos?

In [55]:
### PON AQUÍ TU CÓDIGO EN CASO DE QUE SEA NECESARIO HACER ALGÚN TRATAMIENTO ADICIONAL

#### Media móvil

In [70]:
# Vamos a aplicar la técnica de detección de anomalías de Media Móvil sobre la variable INUT

In [3]:
# Calcula la media móvil

In [4]:
# Define un umbral para detectar anomalías

In [ ]:
# Visualiza los resultados

#### Z-Score

In [68]:
# Vamos a aplicar la técnica de detección de anomalías Z-Score
# Pon aquí tu código

In [ ]:
# Define un umbral para detectar anomalías (por ejemplo, 3 desviaciones estándar)

In [ ]:
# Visualiza los resultados

#### Isolation Forest

In [57]:
# Vamos a aplicar la técnica de detección de anomalías de Isolation Forest, configura el parámetro contamination en 5%
# Pon aquí tu código

In [71]:
# Agrega las columnas de puntaje y si es dato anómalo o no en el dataframe

In [72]:
# Crea un gráfico de barras para mostrar los datos anómalos vs los datos normales

In [73]:
# Crea un gráfico donde se puedan ver los datos anómalos sobre la variable INUT.

In [74]:
# Repite los pasos anteriores con el parámetro de contamination en 1%

## Local Outlier Factor

In [78]:
# Vamos a aplicar la técnica de detección de anomalías de Local Outlier Factor
# Pon aquí tu código

In [79]:
# Crea un gráfico de barras para mostrar los datos anómalos vs los datos normales

In [80]:
# Crea un gráfico donde se puedan ver los datos anómalos sobre la variable INUT.

## Análisis

In [81]:
# Realiza un análisis detallado de los resultados obtenidos en cada una de las técnicas aplicadas. Utiliza gráficos o trablas para apoyar tu conclusiones

## Artículos de investigación

#### Artículo 1

## Articulo: Anomaly Detection for IoT Time-Series Data: A Survey
### **Autores**: Andrew Cook, Goksel Mısırlı, and Zhong Fan
### **Fecha**: 06 Diciembre 2019

### Objetivo
El principal objetivo de la investigación es proporcionar un marco de los desafíos que se pueden encontrar al aplicar técnicas de detección de anomalías a los datos de IoT. Esta investigación también busca discutir una variedad de enfoques que se han desarrollado en diversos dominios, no solo limitados al Internet de las Cosas, y resumir los desafíos actuales en el campo de la detección de anomalías para identificar oportunidades futuras de investigación. 

La investigación se enfoca en desarrollar métodos automatizados para la detección de anomalias en IoT, lo que permite que expertos puedan investigar solo los eventos más importantes en las observaciones, ya que el análisis manual se vuelve inviable a medida que aumenta la complejidad del volumen de datos. Por lo que estas técnicas buscan convertir los datos en crudos en información procesable para nosotros. 

Concretamente el artículo habla sobre la detección de anomalías en diferentes dominios: 

- ***Aplicación general en IoT***: Identificación de observaciones atípicas que pueden afectar futuros análisis realizados sobre datos recopilados en la redes IoT.

- ***IoT Industrial (IIoT) e Industria 4.0***:
  - Monitorización del estado del sistema en la generación de energía a gran escala.
  - Programación inteligente del mantenimiento en plantas de producción más pequeñas.
  - Detección de fallos en sistemas de calefacción, ventilación y aire acondicionado (HVAC) residenciales.
  - Técnicas de control de calidad en la manufactura.
  - Identificación de modos de fallo específicos en maquinaria basada en motores.
  - Predicción y diagnóstico de fallos en turbinas eólicas, permitiendo programar mantenimiento antes de que ocurran problemas.
  -  Detección temprana de fallos en la maquinaria industrial mediante el monitoreo de componentes internos con sensores de audio, reduciendo el tiempo de inactividad inesperado
  - Gestión de concentraciones químicas y de partículas en tanques de plantas de tratamiento de agua, identificando automáticamente estados inadecuados para activar medidas reactivas automáticas.
  - Tener una mejor comprensión del estado operativo de equipos industriales más antiguos a traves de la adaptación de soluciones de monitoreo.


- ***Energía Inteligente (Smart Energy)***:
  - Identificación de fallos en la red de distribución local (como ser, cortes de energía) en tiempo casi real, basándose en los datos de contadores inteligentes.
  - Localización de fallos en la red de distribución, como fallas eléctricas o cambios inesperados de impedancia.
  - Detección de pérdidas no técnicas, como el robo de energía, utilizando información de contadores inteligentes.

- ***Ciudades y Edificios Inteligentes (Smart City and Buildings)***:
  - Identificación de condiciones anómalas en la carretera y ubicaciones de alta congestión utilizando datos de diversas fuentes de dispositivos móviles, lo que permite sugerir rutas alternativas.
  - Monitoreo de la salud de la superficie de las carreteras para programar un mantenimiento oportuno y reducir daños a vehículos o incidentes de tráfico.
  - Mapeo de la congestión en áreas urbanas utilizando datos de trayectoria de autobuses para apoyar la gestión urbana basada en datos.
  - Detección y monitoreo de los niveles de contaminación del aire en entornos urbanos para ayudar a la alcadia a tomar decisiones informadas.
  - Análisis y mejora de la eficiencia energética en edificios comerciales y residenciales.
  - Monitoreo de actividades en situaciones de vida asistida (hogares inteligentes), en donde se aprende el comportamiento "normal" de una persona y se trata de identificar las desviaciones significativas (por ejemplo, una caída) como anomalías para alertar a los profesionales de salud encargados del paciente.  


---

### Técnicas empleadas

Debido a que el artículo es una encuesta de las diferentes técnicas de detección de anomálias que se encuentran en la industria, tenemos una revisión de diferentes técnicas aplicadas a diversas situaciones basado en las caractetisticas de los datos y el problema que se estaba trabajando. Por lo que se resumira por áreas las diferentes técnicas empleadas y el problema al que se aplicaron: 

1. **Técnicas Estadísticas y Probabilísticas**:
- Hicieron uso de modelos ARMA (Autoregressive Moving Average) para datos de series de tiempo, incluyendo ARIMA (Autoregressive Integrated Moving Average) para datos no estacionarios y SARMA (Seasonal ARMA) para estacionalidad.
- Se realizó un análisis de consumo de electricidad en oficinas universitarias con un umbral de 2σ sobre el error de predicción, usando ARIMA y MLP (Multi-Layer Perceptron).
- Detección de fallos y fallas en redes eléctricas usando datos de unidades microsincrofasoriales (sensores de voltaje y corriente)
2.  **Coincidencia de Patrones**: En este caso se visualizan las anomálias etiquetadas y los detectores comparan nuevas observaciones con una base de datos de eventos anomálos. Si no encuentra una coincidencia entonces aprender patrones normales o historicos que sirven de base para proximas detecciones. 
3. **Metodos Basados en Distancias**: Para estos modelos se define una métrica de distancia que sirve para comparar la observación basado en el historico de las mismas. 
4. **Agrupamiento (Clustering)**:
   - El articulo referencia la comparación de distintos métodos de agrupamiento como Isolation Forest junto con ORCA, One-class SVM, LOF y Random Forest para la detección de anomálias. En este caso se comparan los diferentes metodos y debido a la eficiencia computacional que presenta Isolation Forest ven que se puede aplicar este método de detección para datos de flujo (streaming)
   - Se menciona el método de MKAD (Multiple Kernel Anomaly Detection) el cual utiliza funciones kernel para aprender medidas de similitud entre variables del flujo de datos y se compara junto con modelos ORCA y SequenceMiner para la detección de anomálias en datos aeroespaciales. El cual obtiene una mejora significativa en la capacidad de detección tanto en datos continuos como discretos. 
  

Adicionalmente a los métodos probabilisticos y basado en agrupamiento, el articulo menciona modelos predictivos utilizando redes neuronales para la detección de anomalías. En este caso comentamos los modelos estudiados y sus diferentes casos de uso basado en la naturaleza de los datos: 
  - Predicción y diagnóstico de fallos en turbinas eólicas utilizando datos del sistema SCADA.
  - **Autoencoders (AE)**: Reconstruyen el vector de entrada; un error de reconstrucción alto indica una anomalía. Utilizados en dispositivos de sensores con recursos limitados, y para detectar fallos en maquinaria industrial a partir de datos de audio y datos de energía de edificios.
  - **Redes Neuronales Recurrentes (RNNs)**, incluyendo Long Short-term Memory (LSTM) y Gated Recurrent Units (GRUs): Capturan relaciones temporales entre observaciones. Fue utilizado para los siguientes casos:
    - Detección de anomalías en señales ECG (electrocardiograma) usando LSTM.
    - Predicción de valores de series de tiempo con LSTM para detección de anomalías donde una gran pérdida indica una anomalía.
    - Detección de anomalías colectivas en series de tiempo.
    - Detección de anomalías en datos de vuelo de aeronaves usando LSTM y GRU.
    -  Detección de anomalías en telemetría de naves espaciales usando LSTM.
    - Análisis de errores de predicción de RNN con modelos Gaussian Naive-Bayes.
   - Greenhouse method para detección de anomalías fuera de línea en series de tiempo usando RNNs predictivas de múltiples pasos. 
   - Redes Neuronales Bayesianas que usan LSTM y MLP para predicción con nivel de certeza.
   - Simple Online Regression Anomaly Detector (SORAD) para detección de anomalías en línea.
  - **Redes Generativas Antagónicas (GANs)** **con LSTMs** para capturar la naturaleza temporal del sistema y entrenar al discriminador para detectar anomalías en sistemas ciberfísicos con datos de sensores y actuadores
  - **Análisis de Componentes Principales (PCA)**: Descompone un sistema multivariado en un conjunto reducido de variables independientes. Utilizado para anomalías en el tráfico de red y en sistemas IoT con sensores. También se utilizó para reducir la dimensionalidad de datos multivariados antes de alimentar un GAN.
  - **Projection Pursuit**: Reduce la dimensionalidad de un sistema complejo a una dimensión donde se pueden aplicar métodos univariados
  - **Hierarchical Temporal Memory (HTM)**: Modelo bio-inspirado para procesar series de tiempo, resistente al ruido y con aprendizaje continuo en línea. Aplicado a la previsión de carga eléctrica y detección de anomalías en tiempo real.
  -**Dynamic Bayesian Networks (DBN)**: Aplicadas a datos ambientales univariados y multivariados para detección de anomalías en tiempo real, mejorando la detección con la combinación de múltiples flujos de sensores


---

### Conclusiones y Resultados

Nuevamente debido a que el articulo es una encuesta sobre las diferentes técnicas de detección de anomalías en datos de series temporales de IoT entonces se identificaron diferentes desafios en el tratamiento de los mismos: 
- **Información Contextual**: La inclusión de información contextual (temporal, espacial, externa) puede mejorar la detección, pero aumenta la complejidad del proceso.
-**Dimensionalidad**: Los datos pueden ser univariados (de un solo sensor) o multivariados (de múltiples sensores). La detección en flujos multivariados es más compleja ya que no solo importa el historial, sino también la relación entre las mediciones en cada paso de tiempo. La alta dimensionalidad afecta la elección del método y el costo computacional se vuelve un problema rapidamente.
- **Ruido**: Los sistemas IoT a menudo producen datos ruidosos, inconsistentes, faltantes o duplicados, especialmente con sensores de bajo costo y recursos limitados. Reducir el ruido de manera tradicional no siempre genera buenas resultados, ya que un cambio en el patrón del ruido podría ser una anomalía significativa.
- **Datos no estacionarios**: Las series temporales del mundo real suelen ser no estacionarias, lo que significa que la media, la varianza y la autocorrelación cambian con el tiempo. Esto incluye el "Concept Drift" (cambio en la distribución estadística de los datos), la "Estacionalidad" (cambios cíclicos) y los "Puntos de Cambio" (cambios abruptos permanentes). La capacidad de un método para adaptarse a estos cambios es crucial para despliegues en el largo plazo.
- A menudo, no se dispone de conjuntos de datos históricos suficientes o etiquetados que cubran todos los tipos de anomalías posibles, lo que dificulta el uso de enfoques de aprendizaje supervisado tradicionales.
- Se asume que la mayoría de las observaciones son "normales". Los métodos no supervisados o semi-supervisados son más viables, ya que se entrenan con datos "normales" y reportan datos que caen fuera de los límites de normalidad.

Al entender los diferentes desafios presentados para el manejo de datos en IoT los investigadores presentan los siguientes resultados según los diferentes modelos utilizados: 
1. **Modelos Estadísticos y Probabilísticos**:

- ARMA (Autoregressive Moving Average): Construye un modelo paramétrico de la serie de tiempo. Se ha utilizado ampliamente, pero tiene dificultades con conjuntos de datos no estacionarios, especialmente con estacionalidad o cambios de media.
- ARIMA (Autoregressive Integrated Moving Average): Está diseñado para manejar la no estacionariedad mediante la diferenciación de los datos para hacerlos más estacionarios. Se ha combinado con éxito con perceptrones multicapa (MLP) para predecir el consumo de electricidad y detectar anomalías, aunque se identificó una gran sensibilidad a usos ocasionales que requerían reglas adicionales para evitar falsos positivos.
- SARMA (Seasonal ARMA): Aborda la estacionalidad al generar múltiples modelos a través de diferentes desfases temporales estacionales y aplicar las técnicas de ARMA.
2.  **Agrupamiento (Clustering)**:
  
-  Isolation Forests: Se concluyó que, debido a su eficiencia computacional, este enfoque podría aplicarse a datos de flujo (streaming data).
-  Detección de Anomalías con Múltiples Kernels (MKAD): MKAD mostró mejoras significativas en la capacidad de detección en flujos de datos discretos y continuos en comparación con otros métodos contemporáneos.

3. **Enfoques Predictivos**:

- Redes Neuronales Recurrentes (RNNs) como LSTM y GRU han demostrado una gran capacidad para modelar la variabilidad en sistemas univariados complejos. Se utilizan para predecir valores futuros, y el error de reconstrucción o predicción se usa para identificar anomalías. Algunos enfoques permiten el aprendizaje en línea y la adaptación al "concept drift".
- Autoencoders: Métodos como Squeezed CNN-VAE (SCVAE) son más adecuados para dispositivos de borde en IoT debido a su tamaño, complejidad y menor costo de entrenamiento.
- Los enfoques supervisados, aunque útiles para anomalías conocidas, pueden tener dificultades para identificar modos de anomalía novedosos para los que no fueron entrenados.

Basado en estos resultados y hallazgos los investigadores concluyen que no existe un "mejor" enfoque único para la detección de anomalias para datos en IoT debido a la caracteristica de los mismos y que son muy dependientes del entorno de despligue en donde actuen los detectores. Adicionalmente destacan la gran necesidad de que el aprendizaje de los detectores se haga en linea ya que como los datos son no estacionarios y poseen series temporales entonces esto requiere que los métodos se adapten constantemente a los cambios en la distribución de los datos. 

Los investigadores destacan la importancia del uso de datos multivariados que si bien poseen mayor complejidad estos son necesarios para detectar los cambios en las relaciones entre las variables y asi obtener una visión más completa de los sistemas complejos monitoreados en IoT. 




---

In [ ]:
# Coloca en este espacio el análisis del artículo 1

#### Artículo 2

In [ ]:
# Coloca en este espacio el análisis del artículo 2